In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
np.random.seed(42)

# Define breathing pattern parameters
breathing_patterns = {
    "Normal": {"amplitude_range": (0.5, 1.5), "frequency_range": (12, 20)},   # 12-20 BPM
    "Deep": {"amplitude_range": (1.5, 3.0), "frequency_range": (8, 12)},      # 8-12 BPM
    "Shallow": {"amplitude_range": (0.1, 0.5), "frequency_range": (20, 30)},  # 20-30 BPM
    "Apnea": {"amplitude_range": (0.0, 0.1), "frequency_range": (0, 5)},      # Near 0
    "Irregular": {"amplitude_range": (0.1, 3.0), "frequency_range": (5, 30)}  # Mixed
}

# Generate synthetic dataset
n_samples = 2000
timestamps = np.round(np.linspace(0.01, n_samples / 100, n_samples), 2)  # Simulated 0.01s intervals

# Allocate data
data = {
    "Timestamp": [],
    "Sensor Value": [],
    "Amplitude": [],
    "Frequency": [],
    "Label": []
}

for label, params in breathing_patterns.items():
    # Number of samples per breathing type
    n_label_samples = n_samples // len(breathing_patterns)
    
    amplitudes = np.random.uniform(params["amplitude_range"][0], params["amplitude_range"][1], n_label_samples)
    frequencies = np.random.uniform(params["frequency_range"][0], params["frequency_range"][1], n_label_samples)
    sensor_values = np.random.randint(400, 600, n_label_samples)  # Example sensor values

    # Append to data
    data["Timestamp"].extend(timestamps[:n_label_samples])
    data["Sensor Value"].extend(sensor_values)
    data["Amplitude"].extend(amplitudes)
    data["Frequency"].extend(frequencies)
    data["Label"].extend([label] * n_label_samples)

# Create DataFrame
df = pd.DataFrame(data)

# Save dataset to CSV
file_path = "breathing_dataset.csv"
df.to_csv(file_path, index=False)

print(f"Dataset generated and saved as {file_path}")

# Load the dataset
data = pd.read_csv("breathing_dataset.csv")

# Convert the labels to one-hot encoding
labels = pd.get_dummies(data['Label']).values

# Normalize amplitude and frequency for input features
features = data[['Amplitude', 'Frequency']].values
features = (features - features.min(axis=0)) / (features.max(axis=0) - features.min(axis=0))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Reshape the data to fit ResNet input requirements (e.g., 224x224x3)
def preprocess_for_resnet(data):
    reshaped_data = np.zeros((data.shape[0], 224, 224, 3))
    for i in range(data.shape[0]):
        reshaped_data[i, :, :, 0] = data[i, 0]  # Amplitude as red channel
        reshaped_data[i, :, :, 1] = data[i, 1]  # Frequency as green channel
        reshaped_data[i, :, :, 2] = 0           # Zero for blue channel
    return reshaped_data

X_train_resnet = preprocess_for_resnet(X_train)
X_test_resnet = preprocess_for_resnet(X_test)

# Load pre-trained ResNet152 model without top layers
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers for classification
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(labels.shape[1], activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32

history = model.fit(
    X_train_resnet, y_train,
    validation_data=(X_test_resnet, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

# Save the trained model
model.save("resnet152_breathing_model.h5")

# Evaluate the model
loss, accuracy = model.evaluate(X_test_resnet, y_test, verbose=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Load the trained model for predictions
trained_model = load_model("resnet152_breathing_model.h5")

# Function for user input and prediction
def predict_breathing_pattern(amplitude, frequency):
    # Normalize input
    amplitude_normalized = (amplitude - features[:, 0].min()) / (features[:, 0].max() - features[:, 0].min())
    frequency_normalized = (frequency - features[:, 1].min()) / (features[:, 1].max() - features[:, 1].min())

    # Prepare input for ResNet
    input_data = np.zeros((1, 224, 224, 3))
    input_data[0, :, :, 0] = amplitude_normalized
    input_data[0, :, :, 1] = frequency_normalized

    # Predict
    prediction = trained_model.predict(input_data)
    label_index = np.argmax(prediction)
    label_map = {i: label for i, label in enumerate(pd.get_dummies(data['Label']).columns)}

    return label_map[label_index]

# Example usage
amplitude_input = float(input("Enter amplitude value: "))
frequency_input = float(input("Enter frequency value: "))
predicted_label = predict_breathing_pattern(amplitude_input, frequency_input)
print(f"Predicted Breathing Pattern: {predicted_label}")

# Plot accuracy and loss
def plot_training_history(history):
    # Plot accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function to plot
plot_training_history(history)

Dataset generated and saved as breathing_dataset.csv
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2112 - loss: 7.3392   